In [ ]:
#Instalacion de librerias faltantes
!pip install googlemaps
!pip install geopy


Restaurant 2024

In [ ]:
#Librerias
import pandas as pd
import numpy as ny
import googlemaps
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut, GeocoderInsufficientPrivileges
from geopy.extra.rate_limiter import RateLimiter
import logging
import pandas as pd
import time
import re
import csv
import os
import hashlib


Realizamos un  primer análisis para ver la composición de las colunas y de los datos en el archivo

In [ ]:
# 1. Cargamos los datos
file_path = r'/content/Restaurant 2024.csv'
data = pd.read_csv(file_path, delimiter='\t')

# 2. Realizamos una revisión general
print("Dimensiones del dataset:", data.shape)
print("\nNombres de las columnas:", data.columns.tolist())
print("\nPrimeras filas del dataset:")
print(data.head())

# 3. Mostramos la calidad de los datos
print("\nValores nulos por columna:")
print(data.isnull().sum())

print("\nTipos de datos:")
print(data.dtypes)

# 4. Realisamos una estadísticas descriptivas
print("\nEstadísticas descriptivas para columnas numéricas:")
print(data.describe())

print("\nValores únicos por columna:")
print(data.nunique())


Dimensiones del dataset: (10294, 11)

Nombres de las columnas: ['name', 'addr:street', 'addr:city', 'addr:state', 'addr:postcode', 'smoking', 'wheelchair', 'cuisine', 'reviews', '@lat', '@lon']

Primeras filas del dataset:
                  name   addr:street addr:city addr:state addr:postcode  \
0         Smokey Bones           NaN       NaN        NaN           NaN   
1           Wing House  Kirkman Road   Orlando        NaN         32811   
2         Ruby Tuesday           NaN       NaN        NaN           NaN   
3              Denny's           NaN       NaN        NaN           NaN   
4  Spaghetti Warehouse           NaN       NaN        NaN           NaN   

  smoking wheelchair   cuisine  reviews       @lat       @lon  
0      no        NaN  barbecue      NaN  28.550990 -81.203255  
1     NaN        yes       NaN      NaN  28.503706 -81.458043  
2     NaN        yes  american      NaN  28.503323 -81.458029  
3     NaN        NaN  american      NaN  28.502807 -81.458022  
4     

Cargamos datos faltantes en basandonos en la longitud y latitud

In [ ]:
# Configuramos el geolocalizador con un user_agent único
geolocator = Nominatim(user_agent="mi_proyecto_geocodificacion_v1")

# Función para realizar la geocodificación inversa
def reverse_geocode(lat, lon):
    try:
        location = geolocator.reverse((lat, lon), timeout=10)
        if location:
            # Extraemos  dirección completa y ciudad
            address = location.address
            city = location.raw.get('address', {}).get('city', None)
            return address, city
        else:
            return "Dirección no encontrada", None
    except GeocoderTimedOut:
        return "Tiempo de espera agotado", None
    except GeocoderInsufficientPrivileges:
        return "Bloqueado: Verificar API o User-Agent", None

# Actualizamos las columnas addr:street y addr:city
def update_address(row):
    if pd.notna(row['@lat']) and pd.notna(row['@lon']):
        address, city = reverse_geocode(row['@lat'], row['@lon'])
        row['addr:street'] = address  # Guardar dirección completa
        row['addr:city'] = city       # Actualizar ciudad
    return row

# Aplicamos una pausa de 2 segundos fila por fila con retraso para respetar los límites de solisitudes
for index, row in data.iterrows():
    data.loc[index] = update_address(row)
    time.sleep(2)

# Guardamos los resultados en un nuevo archivo CSV
data.to_csv("/content/Restaurant 2024.csv", index=False)

# Mostramos las primeras filas actualizadas
print(data[['@lat', '@lon', 'addr:street', 'addr:city']].head())

        @lat       @lon                                        addr:street  \
0  28.550990 -81.203255  Smokey Bones, Alafaya Trail, Waterford Lakes, ...   
1  28.503706 -81.458043  Wing House, 3789, Kirkman Road, Orlando, Orang...   
2  28.503323 -81.458029  Ruby Tuesday, Kirkman Road, Orlando, Orange Co...   
3  28.502807 -81.458022  Denny's, Kirkman Road, Orlando, Orange County,...   
4  27.961484 -82.445205  Spaghetti Warehouse, East 8th Avenue, Ybor Cit...   

  addr:city  
0      None  
1   Orlando  
2   Orlando  
3   Orlando  
4     Tampa  


Extraemos los datos de la columna direcion

In [ ]:
# Cargamos  los datos
archivo = "/content/Restaurant 2024.csv"
data = pd.read_csv(archivo)

# Renombramos  las columnas
data.rename(columns={
    'addr:street': 'direccion',
    'amenity': 'tipo',
    '@lat': 'latitud',
    '@lon': 'longitud'
}, inplace=True)

# Dividimos la columna direccion para extraer los datos
direccion_split = data['direccion'].fillna('').str.split(',', expand=True)

# Creamos las nuebas colunas a usar
data['nombre'] = direccion_split[0].str.strip()

# Extraemos la direccion combinando todas las partes de la dirección hasta el estado
data['direccion'] = direccion_split.apply(
    lambda row: ', '.join(filter(None, [row[1].strip(), row[2].strip()])), axis=1
)

# Extraemos la ciudad siempre y cuando tenga la palabra County
data['ciudad'] = direccion_split.apply(
    lambda row: next((x.strip() for x in row if 'County' in str(x)), 'No definido'), axis=1
)

# Extraemos el estado  "Florida"
data['estado'] = direccion_split.apply(lambda row: 'Florida', axis=1)

# Extraemos  y validamos los códigos postales
data['codigo_postal'] = direccion_split.apply(
    lambda row: next((re.search(r'\b\d{5}\b', str(x)).group(0) for x in row if re.search(r'\b\d{5}\b', str(x))), 'No definido'), axis=1
)

# Separamos el peis "United States"
data['pais'] = direccion_split.apply(lambda row: 'United States', axis=1)

# Reordenamos las columnas
data = data[['nombre', 'direccion', 'ciudad', 'estado', 'pais', 'codigo_postal', 'longitud', 'latitud']]

# Eliminamos  duplicados
data.drop_duplicates(inplace=True)

# Guardamos el resultado
processed_file_path = '/content/Restaurant 2024.csv'
data.to_csv(processed_file_path, index=False)

# Análisamos de datos vacíos y duplicados
vacios = data.isnull().sum()
duplicados = data.duplicated().sum()

print("Análisis de datos:")
print("Valores vacíos por columna:")
print(vacios)
print(f"Número de filas duplicadas: {duplicados}")
print(data.head())


Análisis de datos:
Valores vacíos por columna:
nombre           0
direccion        0
ciudad           0
estado           0
pais             0
codigo_postal    0
longitud         0
latitud          0
dtype: int64
Número de filas duplicadas: 0
                        nombre                     direccion  \
0                 Ruby Tuesday         Kirkman Road, Orlando   
1                      Denny's         Kirkman Road, Orlando   
2  El Puerto Argentinean Grill  North 17th Street, Ybor City   
3       La Terrazza Restaurant         1727, East 7th Avenue   
4  The Country Club of Florida              22, Country Road   

                ciudad   estado           pais codigo_postal   longitud  \
0        Orange County  Florida  United States         32811 -81.458029   
1        Orange County  Florida  United States         32811 -81.458022   
2  Hillsborough County  Florida  United States         33605 -82.440585   
3  Hillsborough County  Florida  United States         33605 -82.439679  

Eliminamos la palabra County

In [ ]:
# Eliminamos la palabra 'County' de las columnas 'direccion' y 'ciudad'
data['direccion'] = data['direccion'].str.replace(r'\bCounty\b', '', regex=True).str.strip()
data['ciudad'] = data['ciudad'].str.replace(r'\bCounty\b', '', regex=True).str.strip()
print(data[['direccion', 'ciudad']].head())

# Guardamos el resultado
processed_file_path = '/content/Restaurant 2024.csv'
data.to_csv(processed_file_path, index=False)

                      direccion        ciudad
0         Kirkman Road, Orlando        Orange
1         Kirkman Road, Orlando        Orange
2  North 17th Street, Ybor City  Hillsborough
3         1727, East 7th Avenue  Hillsborough
4              22, Country Road    Palm Beach


Creamos la columna id nombre y id ciudad

In [ ]:
# Cargamos las rutas de los  archivos
restaurants_file = '/content/Restaurant 2024.csv'
names_file = '/content/nombres_restaurant.csv'
cities_file = '/content/ciudades_dim.csv'

# Cargamos los archivos
try:
    df_restaurants = pd.read_csv(restaurants_file)
    df_names = pd.read_csv(names_file)
    df_cities = pd.read_csv(cities_file)
except FileNotFoundError as e:
    print(f"Error: {e}")
    raise

# Creaamos IDs únicos para nombres de restaurantes
unique_names = df_restaurants['nombre'].unique()
existing_names = df_names['nombre'].tolist()

# Verificamos y agregar nuevos nombres
new_names = [name for name in unique_names if name not in existing_names]
if new_names:
    new_name_ids = [{'id_nombre': hashlib.md5(name.encode()).hexdigest()[:4], 'nombre': name} for name in new_names]
    df_new_names = pd.DataFrame(new_name_ids)
    df_names = pd.concat([df_names, df_new_names], ignore_index=True)
    df_names.to_csv(names_file, index=False)

# Mapeamos las id_nombres de nombres en df_restaurants
name_to_id_map = pd.Series(df_names['id_nombre'].values, index=df_names['nombre']).to_dict()
df_restaurants['id_nombre'] = df_restaurants['nombre'].map(name_to_id_map)

# Creamos id  únicos para ciudades
unique_cities = df_restaurants['ciudad'].unique()
existing_cities = df_cities['ciudad'].tolist()

# Verificamos y agregamos  nuevas ciudades
new_cities = [city for city in unique_cities if city not in existing_cities]
if new_cities:
    new_city_ids = [{'id_ciudad': hashlib.md5(city.encode()).hexdigest()[:4], 'ciudad': city} for city in new_cities]
    df_new_cities = pd.DataFrame(new_city_ids)
    df_cities = pd.concat([df_cities, df_new_cities], ignore_index=True)
    df_cities.to_csv(cities_file, index=False)

# Mapeamos id_ciudad de ciudades en df_restaurants
city_to_id_map = pd.Series(df_cities['id_ciudad'].values, index=df_cities['ciudad']).to_dict()
df_restaurants['id_ciudad'] = df_restaurants['ciudad'].map(city_to_id_map)

# Reordenamos las columnas
column_order = [
    'id_nombre', 'nombre', 'direccion', 'id_ciudad', 'ciudad', 'estado', 'pais',
    'codigo_postal', 'longitud', 'latitud'
]

# Verificamos y reordenamos las columnas existentes
existing_columns = [col for col in column_order if col in df_restaurants.columns]
df_restaurants = df_restaurants[existing_columns]

# Guardamos el resultado
output_file = '/content/restaurant_2024.csv'
df_restaurants.to_csv(output_file, index=False)

print(f"Archivo procesado y guardado en: {output_file}")


Archivo procesado y guardado en: /content/restaurant_2024.csv


In [ ]:
import pandas as pd

# Ruta del archivo
file_path = '/content/restaurant_2024.csv'

# Cargar el archivo
try:
    df = pd.read_csv(file_path)
except FileNotFoundError as e:
    print(f"Error: {e}")
    raise

# Eliminar la columna 'pais'
if 'pais' in df.columns:
    df = df.drop(columns=['pais'])
    print("Columna 'pais' eliminada correctamente.")
else:
    print("La columna 'pais' no existe en el archivo.")

# Guardar el archivo procesado
output_path = '/content/restaurant_2024.csv'
df.to_csv(output_path, index=False)

print(f"Archivo procesado y guardado en: {output_path}")


Columna 'pais' eliminada correctamente.
Archivo procesado y guardado en: /content/restaurant_2024.csv


Restaurant 2023

In [ ]:
# 1. Cargamos los datos
file_path = r'/content/Restaurant 2023.csv'
data = pd.read_csv(file_path, delimiter='\t')

# 2. Realizamos una revisión general
print("Dimensiones del dataset:", data.shape)
print("\nNombres de las columnas:", data.columns.tolist())
print("\nPrimeras filas del dataset:")
print(data.head())

# 3. Mostramos la calidad de los datos
print("\nValores nulos por columna:")
print(data.isnull().sum())

print("\nTipos de datos:")
print(data.dtypes)

# 4. Realisaos una estadísticas descriptivas
print("\nEstadísticas descriptivas para columnas numéricas:")
print(data.describe())

print("\nValores únicos por columna:")
print(data.nunique())


Dimensiones del dataset: (4487, 11)

Nombres de las columnas: ['name', 'addr:street', 'addr:city', 'addr:state', 'addr:postcode', 'smoking', 'wheelchair', 'cuisine', 'reviews', '@lat', '@lon']

Primeras filas del dataset:
                          name      addr:street addr:city addr:state  \
0                 Ruby Tuesday              NaN       NaN        NaN   
1                      Denny's              NaN       NaN        NaN   
2  El Puerto Argentinean Grill              NaN       NaN        NaN   
3       La Terrazza Restaurant  East 7th Avenue     Tampa         FL   
4  The Country Club of Florida              NaN       NaN        NaN   

  addr:postcode smoking wheelchair   cuisine  reviews       @lat       @lon  
0           NaN     NaN        yes  american      NaN  28.503323 -81.458029  
1           NaN     NaN        NaN  american      NaN  28.502807 -81.458022  
2           NaN     NaN        yes       NaN      NaN  27.958677 -82.440585  
3         33605     NaN        ye

In [ ]:
#Cargamos la ruta del archivo original
archivo = "/content/Restaurant 2023.csv"

# Detectamos el número  de columnas
with open(archivo, 'r') as f:
    header = f.readline().strip()
    expected_columns = len(header.split(','))

correct_lines = []
problem_lines = []

with open(archivo, 'r') as f:
    lines = f.readlines()

for i, line in enumerate(lines):
    if len(line.strip().split(',')) == expected_columns:
        correct_lines.append(line)
    else:
        problem_lines.append((i + 1, line))

# Guardamos archivo limpio
archivo_limpio = "/content/Restaurant_2023_cleaned.csv"
with open(archivo_limpio, 'w') as f:
    f.writelines(correct_lines)

print(f"Archivo limpio guardado en: {archivo_limpio}")
if problem_lines:
    print(f"Se encontraron {len(problem_lines)} líneas problemáticas:")
    for line_info in problem_lines[:5]:
        print(f"Línea {line_info[0]}: {line_info[1]}")

try:
    data = pd.read_csv(archivo_limpio)

    # Renombramos columnas
    data.rename(columns={
        'addr:street': 'direccion',
        'amenity': 'tipo',
        '@lat': 'latitud',
        '@lon': 'longitud'
    }, inplace=True)

    print("Archivo cargado exitosamente. Primeras filas:")
    print(data.head())

except Exception as e:
    print(f"Error al cargar el archivo limpio: {e}")


Archivo limpio guardado en: /content/Restaurant_2023_cleaned.csv
Se encontraron 16 líneas problemáticas:
Línea 618: Komoon Thai Sushi Ceviche	"Bonita Beach Rd, Suite 101"			34134			thai		26.3310766	-81.8104886

Línea 1037: "Ronnie's Wings, Oysters and More"	Walnut Street	Green Cove Springs	FL	32043			wings;oyster;seafood		29.9939195	-81.6782346

Línea 1338: "The Corner Cafe, Home of Gourmet2Go"	W 1st Street	Sanford	FL	32771			sandwich		28.8115870	-81.2682113

Línea 1419: Shindig Irish Restaurant & Pub	"464 SW Port St Lucie Blvd, Suite 115"	Port Saint Lucie	FL	34953			irish;american		27.2716911	-80.3593372

Línea 1420: Altavilla Pizza & Trattoria	"7550 Mission Hills Dr, #318"	Naples	FL	34119			pizza;italian;sandwich		26.2469744	-81.6902155

Archivo cargado exitosamente. Primeras filas:
  name\taddr:street\taddr:city\taddr:state\taddr:postcode\tsmoking\twheelchair\tcuisine\treviews\t@lat\t@lon
0  Ruby Tuesday\t\t\t\t\t\tyes\tamerican\t\t28.50...                                           

Extraemos los datos faltantes en base a la latitud y longuitud

In [ ]:
# Configuramos el geolocalizador con un user_agent único
geolocator = Nominatim(user_agent="mi_proyecto_geocodificacion_v1")

# Función para realizar la geocodificación inversa y buscar el nombre del lugar
def reverse_geocode(lat, lon):
    try:
        location = geolocator.reverse((lat, lon), timeout=10)
        if location:
            address = location.address
            city = location.raw.get('address', {}).get('city', None)
            name = location.raw.get('name', None) or location.raw.get('address', {}).get('attraction', None)
            return address, city, name
        else:
            return "Dirección no encontrada", None, None
    except GeocoderTimedOut:
        return "Tiempo de espera agotado", None, None
    except GeocoderInsufficientPrivileges:
        return "Bloqueado: Verificar API o User-Agent", None, None

# Limpieza inicial del archivo
input_file = "/content/Restaurant 2023.csv"
cleaned_file = "/content/Restaurant 2023 limpio.csv"

with open(input_file, 'r', encoding='utf-8') as infile, open(cleaned_file, 'w', encoding='utf-8') as outfile:
    for line in infile:
        if line.strip():  # Omitir líneas vacías
            outfile.write(line)

# Cargar el archivo limpio
try:
    data = pd.read_csv(cleaned_file, sep='\t', encoding='utf-8')
except pd.errors.ParserError as e:
    print(f"Error al cargar el archivo: {e}")
    data = pd.read_csv(cleaned_file, sep=',', encoding='utf-8', on_bad_lines='skip')

# Añadir la columna 'name' si no existe
if 'name' not in data.columns:
    data['name'] = None

# Actualizar las columnas addr:street, addr:city y name
def update_address(row):
    if pd.notna(row['@lat']) and pd.notna(row['@lon']):
        address, city, name = reverse_geocode(row['@lat'], row['@lon'])
        row['addr:street'] = address
        row['addr:city'] = city
        row['name'] = name
    return row

# Aplicar la función fila por fila con pausas para respetar límites de solicitudes
for index, row in data.iterrows():
    data.loc[index] = update_address(row)
    time.sleep(2)

# Guardar los resultados
output_file = "/content/Restaurant 2023.csv"
data.to_csv(output_file, index=False)

# Mostrar las primeras filas procesadas
print(data.head())

                          name  \
0                 Ruby Tuesday   
1                      Denny's   
2  El Puerto Argentinean Grill   
3       La Terrazza Restaurant   
4  The Country Club of Florida   

                                         addr:street addr:city addr:state  \
0  Ruby Tuesday, Kirkman Road, Orlando, Orange Co...   Orlando        NaN   
1  Denny's, Kirkman Road, Orlando, Orange County,...   Orlando        NaN   
2  El Puerto Argentinean Grill, North 17th Street...     Tampa        NaN   
3  La Terrazza Restaurant, 1727, East 7th Avenue,...     Tampa         FL   
4  The Country Club of Florida, 22, Country Road,...      None        NaN   

  addr:postcode smoking wheelchair   cuisine  reviews       @lat       @lon  
0           NaN     NaN        yes  american      NaN  28.503323 -81.458029  
1           NaN     NaN        NaN  american      NaN  28.502807 -81.458022  
2           NaN     NaN        yes       NaN      NaN  27.958677 -82.440585  
3         33605     

Extraemos los datos de la columna direccion

In [ ]:
# Cargamos  los datos
archivo = "/content/Restaurant 2023.csv"
data = pd.read_csv(archivo)

# Renombramos  las columnas
data.rename(columns={
    'addr:street': 'direccion',
    'amenity': 'tipo',
    '@lat': 'latitud',
    '@lon': 'longitud'
}, inplace=True)

# Dividimos la columna direccion para extraer los datos
direccion_split = data['direccion'].fillna('').str.split(',', expand=True)

# Creamos las nuebas colunas a usar
data['nombre'] = direccion_split[0].str.strip()

# Extraemos la direccion combinando todas las partes de la dirección hasta el estado
data['direccion'] = direccion_split.apply(
    lambda row: ', '.join(filter(None, [row[1].strip(), row[2].strip()])), axis=1
)

# Extraemos la ciudad siempre y cuando tenga la palabra County
data['ciudad'] = direccion_split.apply(
    lambda row: next((x.strip() for x in row if 'County' in str(x)), 'No definido'), axis=1
)

# Extraemos el estado  "Florida"
data['estado'] = direccion_split.apply(lambda row: 'Florida', axis=1)

# Extraemos  y validamos los códigos postales
data['codigo_postal'] = direccion_split.apply(
    lambda row: next((re.search(r'\b\d{5}\b', str(x)).group(0) for x in row if re.search(r'\b\d{5}\b', str(x))), 'No definido'), axis=1
)

# Separamos el peis "United States"
data['pais'] = direccion_split.apply(lambda row: 'United States', axis=1)

# Reordenamos las columnas
data = data[['nombre', 'direccion', 'ciudad', 'estado', 'pais', 'codigo_postal', 'longitud', 'latitud']]

# Eliminamos  duplicados
data.drop_duplicates(inplace=True)

# Guardamos el resultado
processed_file_path = '/content/Restaurant 2023.csv'
data.to_csv(processed_file_path, index=False)

# Análisamos de datos vacíos y duplicados
vacios = data.isnull().sum()
duplicados = data.duplicated().sum()

print("Análisis de datos:")
print("Valores vacíos por columna:")
print(vacios)
print(f"Número de filas duplicadas: {duplicados}")
print(data.head())


Análisis de datos:
Valores vacíos por columna:
nombre           0
direccion        0
ciudad           0
estado           0
pais             0
codigo_postal    0
longitud         0
latitud          0
dtype: int64
Número de filas duplicadas: 0
                        nombre                     direccion  \
0                 Ruby Tuesday         Kirkman Road, Orlando   
1                      Denny's         Kirkman Road, Orlando   
2  El Puerto Argentinean Grill  North 17th Street, Ybor City   
3       La Terrazza Restaurant         1727, East 7th Avenue   
4  The Country Club of Florida              22, Country Road   

                ciudad   estado           pais codigo_postal   longitud  \
0        Orange County  Florida  United States         32811 -81.458029   
1        Orange County  Florida  United States         32811 -81.458022   
2  Hillsborough County  Florida  United States         33605 -82.440585   
3  Hillsborough County  Florida  United States         33605 -82.439679  

<ipython-input-6-8cfcda072972>:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.drop_duplicates(inplace=True)


Eliminamos la palabra County

In [ ]:
# Eliminamos la palabra 'County' de las columnas 'direccion' y 'ciudad'
data['direccion'] = data['direccion'].str.replace(r'\bCounty\b', '', regex=True).str.strip()
data['ciudad'] = data['ciudad'].str.replace(r'\bCounty\b', '', regex=True).str.strip()
print(data[['direccion', 'ciudad']].head())

# Guardamos el resultado
processed_file_path = '/content/Restaurant 2023.csv'
data.to_csv(processed_file_path, index=False)

                      direccion        ciudad
0         Kirkman Road, Orlando        Orange
1         Kirkman Road, Orlando        Orange
2  North 17th Street, Ybor City  Hillsborough
3         1727, East 7th Avenue  Hillsborough
4              22, Country Road    Palm Beach


In [ ]:
# Cargamos las rutas de los  archivos
restaurants_file = '/content/Restaurant 2023.csv'
names_file = '/content/nombres_restaurant.csv'
cities_file = '/content/ciudades_dim.csv'

# Cargamos los archivos
try:
    df_restaurants = pd.read_csv(restaurants_file)
    df_names = pd.read_csv(names_file)
    df_cities = pd.read_csv(cities_file)
except FileNotFoundError as e:
    print(f"Error: {e}")
    raise

# Creaamos IDs únicos para nombres de restaurantes
unique_names = df_restaurants['nombre'].unique()
existing_names = df_names['nombre'].tolist()

# Verificamos y agregar nuevos nombres
new_names = [name for name in unique_names if name not in existing_names]
if new_names:
    new_name_ids = [{'id_nombre': hashlib.md5(name.encode()).hexdigest()[:4], 'nombre': name} for name in new_names]
    df_new_names = pd.DataFrame(new_name_ids)
    df_names = pd.concat([df_names, df_new_names], ignore_index=True)
    df_names.to_csv(names_file, index=False)

# Mapeamos las id_nombres de nombres en df_restaurants
name_to_id_map = pd.Series(df_names['id_nombre'].values, index=df_names['nombre']).to_dict()
df_restaurants['id_nombre'] = df_restaurants['nombre'].map(name_to_id_map)

# Creamos id  únicos para ciudades
unique_cities = df_restaurants['ciudad'].unique()
existing_cities = df_cities['ciudad'].tolist()

# Verificamos y agregamos  nuevas ciudades
new_cities = [city for city in unique_cities if city not in existing_cities]
if new_cities:
    new_city_ids = [{'id_ciudad': hashlib.md5(city.encode()).hexdigest()[:4], 'ciudad': city} for city in new_cities]
    df_new_cities = pd.DataFrame(new_city_ids)
    df_cities = pd.concat([df_cities, df_new_cities], ignore_index=True)
    df_cities.to_csv(cities_file, index=False)

# Mapeamos id_ciudad de ciudades en df_restaurants
city_to_id_map = pd.Series(df_cities['id_ciudad'].values, index=df_cities['ciudad']).to_dict()
df_restaurants['id_ciudad'] = df_restaurants['ciudad'].map(city_to_id_map)

# Reordenamos las columnas
column_order = [
    'id_nombre', 'nombre', 'direccion', 'id_ciudad', 'ciudad', 'estado', 'pais',
    'codigo_postal', 'longitud', 'latitud'
]

# Verificamos y reordenamos las columnas existentes
existing_columns = [col for col in column_order if col in df_restaurants.columns]
df_restaurants = df_restaurants[existing_columns]

# Guardamos el resultado
output_file = '/content/restaurant_2023.csv'
df_restaurants.to_csv(output_file, index=False)

print(f"Archivo procesado y guardado en: {output_file}")


Archivo procesado y guardado en: /content/restaurant_2023.csv


Convertimos a formato parquet

In [ ]:
# Cargamos la ruta del archivo
file_path = '/content/restaurant_2023.csv'

# Cargamos el archivo
try:
    df = pd.read_csv(file_path)
except FileNotFoundError as e:
    print(f"Error: {e}")
    raise

# Guardamos  el archivo en formato Parquet
output_path = '/content/restaurant_2023.parquet'
df.to_parquet(output_path, index=False)

print(f"Archivo convertido y guardado en formato Parquet en: {output_path}")


Archivo convertido y guardado en formato Parquet en: /content/restaurant_2023.parquet


Restaurant 2022

In [ ]:
# 1. Cargamos los datos
file_path = r'/content/Restaurant 2022.csv'
data = pd.read_csv(file_path, delimiter='\t')

# 2. Realizamos una revisión general
print("Dimensiones del dataset:", data.shape)
print("\nNombres de las columnas:", data.columns.tolist())
print("\nPrimeras filas del dataset:")
print(data.head())

# 3. Mostramos la calidad de los datos
print("\nValores nulos por columna:")
print(data.isnull().sum())

print("\nTipos de datos:")
print(data.dtypes)

# 4. Realisaos una estadísticas descriptivas
print("\nEstadísticas descriptivas para columnas numéricas:")
print(data.describe())

print("\nValores únicos por columna:")
print(data.nunique())


Dimensiones del dataset: (5489, 11)

Nombres de las columnas: ['name', 'addr:street', 'addr:city', 'addr:state', 'addr:postcode', 'smoking', 'wheelchair', 'cuisine', 'reviews', '@lat', '@lon']

Primeras filas del dataset:
                          name      addr:street addr:city addr:state  \
0                   Wing House     Kirkman Road   Orlando        NaN   
1                 Ruby Tuesday              NaN       NaN        NaN   
2                      Denny's              NaN       NaN        NaN   
3  El Puerto Argentinean Grill              NaN       NaN        NaN   
4       La Terrazza Restaurant  East 7th Avenue     Tampa         FL   

  addr:postcode smoking wheelchair   cuisine  reviews       @lat       @lon  
0         32811     NaN        yes       NaN      NaN  28.503706 -81.458043  
1           NaN     NaN        yes  american      NaN  28.503323 -81.458029  
2           NaN     NaN        NaN  american      NaN  28.502807 -81.458022  
3           NaN     NaN        ye

Descargamos los archivos faltantes en bace ala latitud y longitud

In [ ]:
# Configuramos el geolocalizador con un user_agent único
geolocator = Nominatim(user_agent="mi_proyecto_geocodificacion_v1")

# Función para realizar la geocodificación inversa
def reverse_geocode(lat, lon):
    try:
        location = geolocator.reverse((lat, lon), timeout=10)
        if location:
            # Extraemos  dirección completa y ciudad
            address = location.address
            city = location.raw.get('address', {}).get('city', None)
            return address, city
        else:
            return "Dirección no encontrada", None
    except GeocoderTimedOut:
        return "Tiempo de espera agotado", None
    except GeocoderInsufficientPrivileges:
        return "Bloqueado: Verificar API o User-Agent", None

# Actualizamos las columnas addr:street y addr:city
def update_address(row):
    if pd.notna(row['@lat']) and pd.notna(row['@lon']):
        address, city = reverse_geocode(row['@lat'], row['@lon'])
        row['addr:street'] = address  # Guardar dirección completa
        row['addr:city'] = city       # Actualizar ciudad
    return row

# Aplicamos una pausa de 2 segundos fila por fila con retraso para respetar los límites de solisitudes
for index, row in data.iterrows():
    data.loc[index] = update_address(row)
    time.sleep(2)

# Guardamos los resultados en un nuevo archivo CSV
data.to_csv("/content/Restaurant 2022.csv", index=False)

# Mostramos las primeras filas actualizadas
print(data[['@lat', '@lon', 'addr:street', 'addr:city']].head())

        @lat       @lon                                        addr:street  \
0  28.503706 -81.458043  Wing House, 3789, Kirkman Road, Orlando, Orang...   
1  28.503323 -81.458029  Ruby Tuesday, Kirkman Road, Orlando, Orange Co...   
2  28.502807 -81.458022  Denny's, Kirkman Road, Orlando, Orange County,...   
3  27.958677 -82.440585  El Puerto Argentinean Grill, North 17th Street...   
4  27.960129 -82.439679  La Terrazza Restaurant, 1727, East 7th Avenue,...   

  addr:city  
0   Orlando  
1   Orlando  
2   Orlando  
3     Tampa  
4     Tampa  


Extraemos la informacion de la columna direcciones

In [ ]:
# Cargamos  los datos
archivo = "/content/Restaurant 2022.csv"
data = pd.read_csv(archivo)

# Renombramos  las columnas
data.rename(columns={
    'addr:street': 'direccion',
    'amenity': 'tipo',
    '@lat': 'latitud',
    '@lon': 'longitud'
}, inplace=True)

# Dividimos la columna direccion para extraer los datos
direccion_split = data['direccion'].fillna('').str.split(',', expand=True)

# Creamos las nuebas colunas a usar
data['nombre'] = direccion_split[0].str.strip()

# Extraemos la direccion combinando todas las partes de la dirección hasta el estado
data['direccion'] = direccion_split.apply(
    lambda row: ', '.join(filter(None, [row[1].strip(), row[2].strip()])), axis=1
)

# Extraemos la ciudad siempre y cuando tenga la palabra County
data['ciudad'] = direccion_split.apply(
    lambda row: next((x.strip() for x in row if 'County' in str(x)), 'No definido'), axis=1
)

# Extraemos el estado  "Florida"
data['estado'] = direccion_split.apply(lambda row: 'Florida', axis=1)

# Extraemos  y validamos los códigos postales
data['codigo_postal'] = direccion_split.apply(
    lambda row: next((re.search(r'\b\d{5}\b', str(x)).group(0) for x in row if re.search(r'\b\d{5}\b', str(x))), 'No definido'), axis=1
)

# Separamos el peis "United States"
data['pais'] = direccion_split.apply(lambda row: 'United States', axis=1)

# Reordenamos las columnas
data = data[['nombre', 'direccion', 'ciudad', 'estado', 'pais', 'codigo_postal', 'longitud', 'latitud']]

# Eliminamos  duplicados
data.drop_duplicates(inplace=True)

# Guardamos el resultado
processed_file_path = '/content/Restaurant 2022.csv'
data.to_csv(processed_file_path, index=False)

# Análisamos de datos vacíos y duplicados
vacios = data.isnull().sum()
duplicados = data.duplicated().sum()

print("Análisis de datos:")
print("Valores vacíos por columna:")
print(vacios)
print(f"Número de filas duplicadas: {duplicados}")
print(data.head())


Análisis de datos:
Valores vacíos por columna:
nombre           0
direccion        0
ciudad           0
estado           0
pais             0
codigo_postal    0
longitud         0
latitud          0
dtype: int64
Número de filas duplicadas: 0
                        nombre                     direccion  \
0                   Wing House            3789, Kirkman Road   
1                 Ruby Tuesday         Kirkman Road, Orlando   
2                      Denny's         Kirkman Road, Orlando   
3  El Puerto Argentinean Grill  North 17th Street, Ybor City   
4       La Terrazza Restaurant         1727, East 7th Avenue   

                ciudad   estado           pais codigo_postal   longitud  \
0        Orange County  Florida  United States         32811 -81.458043   
1        Orange County  Florida  United States         32811 -81.458029   
2        Orange County  Florida  United States         32811 -81.458022   
3  Hillsborough County  Florida  United States         33605 -82.440585  

<ipython-input-8-e160d0cc461c>:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.drop_duplicates(inplace=True)


Eliminamos la palabra County

In [ ]:
# Eliminamos la palabra 'County' de las columnas 'direccion' y 'ciudad'
data['direccion'] = data['direccion'].str.replace(r'\bCounty\b', '', regex=True).str.strip()
data['ciudad'] = data['ciudad'].str.replace(r'\bCounty\b', '', regex=True).str.strip()
print(data[['direccion', 'ciudad']].head())

# Guardamos el resultado
processed_file_path = '/content/Restaurant 2022.csv'
data.to_csv(processed_file_path, index=False)

                      direccion        ciudad
0            3789, Kirkman Road        Orange
1         Kirkman Road, Orlando        Orange
2         Kirkman Road, Orlando        Orange
3  North 17th Street, Ybor City  Hillsborough
4         1727, East 7th Avenue  Hillsborough
